# 装饰器原理

## 定义
装饰器是可调用的对象，其参数是另外一个函数（被装饰的函数），即被装饰的函数是装饰器函数的参数

In [1]:
# 比如有一个名为decorate的函数，那么
@decorate
def target():
    print("running target()")
    
# 和

def target():
    print("running target()")
target = decorate(target)
# 两种写法是一样的

## 装饰器的特性
- 能把被装饰的函数替换成其他函数
- 能在加载模块的时候立即执行

In [10]:
# 把装饰的函数替换成其他函数
def deco(func):
    def inner():
        print("running inner()")
    return inner


@deco
def target():
    print("running target")

    
target()
target
# 可见，通过装饰器，本质上吧target替换成了inner函数

running inner()


<function __main__.deco.<locals>.inner()>

函数装饰器在导入时立即执行，而被装饰的函数只在明确调用时执行。
详细的去看《流畅的python》7.2章吧<br><br>另外，说一下装饰器的常见用法
- 实际情况下，装饰器通常在一个模块中定义，然后应用到其它模块的函数中
- 大多数装饰器都会在内部定义一个函数，然后将其返回。关于这一点有个很重要的知识点：**使用内部函数的代码几乎要靠闭包才能够正常运作，具体如何看下面**

## 装饰器改进策略模式
以上节电商促销为例

In [ ]:
# 装饰器改进最优折扣的实现best_promo，改进忘记在列表中输入新策略的问题
promos = []


def promotion(promo_func):
    promos.append(promo_func)
    return promo_func


@promotion
def fidelity(order): 
    """ 为 积 分 为 1000 或 以 上 的 顾 客 提 供 5% 折 扣""" 
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def best_promo(order)：
    """ 选 择 可 用 的 最 佳 折 扣 """ 
    return max(promo( order) for promo in promos)


# 闭包

## 变量作用域
下面有个有意思的例子。pyhton不要求声明变量，但是假定在函数定义体中，赋值的变量是局部变量。

In [17]:
b = "quanju"
def f1(a):
    print(a)
    print(b)
    b = 6
f1(2)


2


UnboundLocalError: local variable 'b' referenced before assignment

## 闭包
闭包和匿名函数有点像，都是在函数内部定义函数  
闭包指延伸了作用于的函数  
函数是不是匿名没有关系，关键是它能够访问定义体之外定义的非全局变量。

In [24]:
# 求平均的实现，在不断新增数据的情况下
# 方法1，使用类


class Average():
    
    def __init__(self):
        self.series = []
        
    def __call__(self, num):
        self.series.append(num)
        total = sum(self.series)
        return total/len(self.series)
avg = Average()
a1 = avg(10)
a2 = avg(11)
print(a1,a2)

10.0 10.5


In [38]:
# 方法2，使用闭包
def make_average():
    series = []
    def average(num):
        series.append(num)
        total = sum(series)
        return total/len(series)
    return average

avg = make_average()
b1 = avg(10)
b2 = avg(11)
print(b1,b2)

# avg.__closure__
avg.__closure__[0].cell_contents

10.0 10.5


[10, 11]

在average函数中，series是一个自由变量，指没有在本地作用于绑定的局部变量  
闭包是一种函数__closure__，他会保留定义函数时存在的自由变量的绑定，这样在调用函数时，虽然定义的作用域不在了，但是仍然能够使用那些绑定。  
另外，只有嵌套在其他函数内的函数才可能处理不在全局作用域中的外部变量。

## nonlocal声明
nonlocal的作用是把变量标记为自由变量  
对于数字，字符串，元组等是不可变类型，只能读取，不能更新  
如果series不是列表而是数字的话，那么如果在闭包实现series += 1，那么就会隐式创建局部变量，这样它就不是自由变量了，就实现不了闭包了。  
global可以从嵌套的作用域开始执行，但是nonlocal所声明的变量必须已经存在，不然会报错

In [2]:
def make_averager(): 
    count = 0 
    total = 0 
    def averager( new_value): 
        nonlocal count, total 
        count += 1 
        total += new_value 
        return total/count 
    return averager

# 实现装饰器
## 简单的装饰器

In [4]:
# 实现输出函数的运行时间
import time 

def clock(func): 
    def clocked(*args):
        t0 = time.perf_counter() 
        # perf_counter()用来计算程序运行时间的
        result = func(*args) 
        elapsed = time.perf_counter() - t0 
        name = func.__name__ 
        # __name__就是用来获取字符串的函数名称
        arg_str = ', '.join(repr(arg) for arg in args) 
        # repr返回一个对象的字符串类型
        print('[%0.8fs]%s(%s) -> %r' % (elapsed, name, arg_str, result)) 
        return result 
    return clocked 




'main'